# Importar librerías a utilizar

In [ ]:
!pip uninstall scikit-learn
!pip install scikit-learn
!pip install contexto

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Would remove:
    /usr/local/lib/python3.9/dist-packages/scikit_learn-1.2.2.dist-info/*
    /usr/local/lib/python3.9/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.9/dist-packages/sklearn/*
Proceed (Y/n)? Y
  Successfully uninstalled scikit-learn-1.2.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
Failed to build scikit-learn
  error: subprocess-exited-with-error
  
  × Running s

In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_friedman1
from sklearn.decomposition import SparsePCA

import sklearn
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')

import requests 
import time
from tqdm import tqdm
from multiprocessing.dummy import Pool
import pandas as pd
import random
from datetime import datetime
import pytz
import re
import string
import numpy as np


from contexto.limpieza import *
from nltk.stem import PorterStemmer
import nltk
import time
from contexto.stemming import Stemmer, stem_texto

import time
from contexto.lematizacion import LematizadorSpacy, LematizadorStanza
from contexto.lematizacion import lematizar_texto

from contexto.exploracion import obtener_ngramas
from contexto.exploracion import nube_palabras
from contexto.exploracion import par_nubes

from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import os
import pandas as pd

# Cargar bases de datos de entrenamiento y testeo

In [ ]:
train = pd.read_csv(r"/content/train.csv")
test = pd.read_csv(r"/content/test.csv")

In [ ]:
train.head() # Ver los primeros 5 tweets de la base de entrenamiento

,id,name,text
0,5f993ca6ab24ecb07efc5259,Petro,Me gustaría que esta columna fuera deliberada ...
1,f8734ee208059bc34690fbe8,Lopez,¡Si Bogotá y Cundinamarca salen adelante Colom...
2,cfd3678247b29ea7ba1fbe3c,Lopez,Esto significa el proyecto de acuerdo de resca...
3,e13d9e299ac5ffd38f503c5a,Lopez,¡Avanza nuestro recorrido por San Cristóbal! \...
4,fc457807261de42e57f5c1c5,Petro,Miles de personas en Bucaramanga en estos mome...


# Limpiar tweets de las bases de entrenamiento y testeo

In [ ]:
# Convertir en lista cada una de las columnas de la base de entrenamiento y testeo
text_train ={'id': train['id'].to_list(),'name': train['name'].to_list(), 'text': train['text'].to_list()}
text_test ={'id': test['id'].to_list(), 'text': test['text'].to_list()}

In [ ]:
# Definir la lista de stopwords
stopwords = []
lis = [lista_nombres()[0], lista_apellidos()[0],lista_geo_colombia()[0], lista_geo_colombia()[1], 
       lista_stopwords()]
for i in lis:
    stopwords.extend(i)

In [ ]:
# Crear función para limpiar el texto
def limpiar(text):
    text = remover_stopwords(text, lista_palabras= stopwords)
    text = re.sub(r"http\S+", "", text) # Eliminar hipervínculos
    text = quitar_repetidos(text, ",") # Eliminar palabras que, estando una coma, están repetidas
    text = limpieza_basica(text) # Eliminar signos de puntuación, números y transformar todo a minúscula
    text = remover_acentos(text) # Eliminar acentos
    text = remover_palabras_cortas(text, 4) # Eliminar palabras que tengan 4 caracteres o menos
    return text

In [ ]:
# Aplicar la función de limpieza creada y aplicar stemmer y lematizador a la base de entrenamiento
re_limp=[]

for i in tqdm(text_train['text']):
    re_limp.append(limpiar(i))

# Stemmer
stemmer = Stemmer(lenguaje='español')

corpus_stem=[]
for i in tqdm(re_limp):
    stem = stem_texto(i, stemmer=stemmer)
    corpus_stem.append(stem)

train_stem = train.drop(['text'], axis=1)
train_stem['text'] = corpus_stem

# Lematizador
lematizador = LematizadorSpacy('es')

corpus_lem =[]
for i in tqdm(re_limp):
    lem = lematizar_texto(i, lematizador=lematizador)
    corpus_lem.append(lem)

train_lem = train.drop(['text'], axis=1)
train_lem['text'] = corpus_lem

train_lem

100%|██████████| 9349/9349 [00:42<00:00, 217.78it/s]


,id,name,text
0,5f993ca6ab24ecb07efc5259,Petro,gustario columna deliberado punto resistencia ...
1,f8734ee208059bc34690fbe8,Lopez,salir adelante adelante trabajar junto reconoc...
2,cfd3678247b29ea7ba1fbe3c,Lopez,este significar proyecto acuerdo rescate socia...
3,e13d9e299ac5ffd38f503c5a,Lopez,avanzar recorrido cristobal llegar colegio ter...
4,fc457807261de42e57f5c1c5,Petro,mil persona momento calle paronacional termina...
...,...,...,...
9344,a9bbd1f1a75509dfbfd58989,Uribe,apoyar construccion politico jovenes austerida...
9345,5444d8088d86cbb383ea478c,Petro,educacion superior duda clave disminucion desi...
9346,ea1e50eeb549aa47ea59a9db,Uribe,jovenes anos desempleado trofimoff foro centro...
9347,7746b5b0907344492c91201e,Lopez,material ir recuperar vida perder cambio doler...


## Testeo

In [ ]:
# Aplicar la función de limpieza creada y aplicar stemmer y lematizador a la base de testeo
re_limp=[]

for i in tqdm(text_test['text']):
    re_limp.append(limpiar(i))

## Stemmer
stemmer = Stemmer(lenguaje='español')

corpus_stem=[]
for i in tqdm(re_limp):
    stem = stem_texto(i, stemmer=stemmer)
    corpus_stem.append(stem)

test_stem = test.drop(['text'], axis=1)
test_stem['text'] = corpus_stem

## Lematizador
lematizador = LematizadorSpacy('es')

corpus_lem =[]
for i in tqdm(re_limp):
    lem = lematizar_texto(i, lematizador=lematizador)
    corpus_lem.append(lem)


test_lem = test.drop(['text'], axis=1)
test_lem['text'] = corpus_lem

test_lem

100%|██████████| 1500/1500 [00:08<00:00, 184.24it/s]


,id,text
0,6f4f09622255e9d9d76251b4,ruego procuradora general diferenciar alcaldio...
1,aefd318f57227e4053f64191,millón peso invertido ipial instalacion acomet...
2,d1eb53be99fa263c10911af9,luchar crisis climatico implicar cambiar movil...
3,bb65250f1ff7582e4380a9c9,opinionescruzada rcnradio rete publicamente qu...
4,0017570ee84b80e1c3e10218,ante amenaza sistematico reduccion fuerza poli...
...,...,...
1495,4d1dc06d6ca8d0da93ceb42a,pedir perdon periodico debate senadoro paola h...
1496,db282aa075d8ffb4dc9d72ae,estilo politico visión pai odio clase destrucc...
1497,afaac0a39ec1ea5f58308407,urgente informar companera pueblo embera katio...
1498,262b591834494af862ed6f09,empresa avicola poder sacar huevo entrar alime...


In [ ]:
# Base de entrenamiento
train_tweets = train_lem['text'].tolist()
train_labels = train_lem['name'].tolist()

# Base de testeo
test_tweets = test_lem['text'].tolist()

In [ ]:
cv = CountVectorizer()
cv.fit_transform(train_tweets + test_tweets)

<10849x18117 sparse matrix of type '<class 'numpy.int64'>'
	with 159157 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf = TfidfTransformer()

train_features = tfidf.fit_transform(cv.transform(train_tweets))
test_features = tfidf.transform(cv.transform(test_tweets))

In [ ]:
print("En entrenamiento, el número de filas son:", train_features.shape[0], "y el número de columnas son: ", train_features.shape[1])

En entrenamiento, el número de filas son: 9349 y el número de columnas son:  18117


In [ ]:
print("En testeo, el número de filas son:", test_features.shape[0], "y el número de columnas son: ", test_features.shape[1])

En testeo, el número de filas son: 1500 y el número de columnas son:  18117


In [ ]:
train_data, test_data = train_test_split(train_lem, test_size=0.2, random_state=42)

In [ ]:
train_data

,id,name,text
6443,fa93b1f78b7c3887a5c18b33,Petro,ataque caudio electoral trapero quitar ojo jov...
5846,d992ebc2dab434d7ed625298,Lopez,juntoscuidamosbogota
6972,717bcfeedf544aec375fa254,Lopez,querer cambiar él cara bogota actor sistema di...
251,d7209aab3fa9cb67c45057d6,Uribe,dias intervencion integral fuerzapublico logro...
6950,7059c1122bdbb937b88d0716,Petro,minuto entrevistar gustavo gomez caracolradio
...,...,...,...
5734,41b9786eec413d9980cba113,Lopez,este contrato social jovenesalau apostar cumpl...
5191,0ee30ae87235377ba0ef9ee6,Uribe,dejar leer entrevista yaya blondie andres muje...
5390,55536c9007d96a131e4b7e88,Petro,captainpolombia ivanducar este gente representar
860,de130ff13da2f4f4aa1823ef,Petro,interés especulacion inmobiliario llevar const...


# Crear base de entrenamiento

Para tener una proxy del desempeño de los modelos planteados, se crea una base de testeo a partir de la base de entrenamiento. Sobre esta base se realiza una primera evaluación para ver el desempeño del modelo antes de subirlo a la plataforma Kaggle.

In [ ]:
# Base de entrenamiento
train_tweets1 = train_data['text'].tolist()
train_labels1 = train_data['name'].tolist()

# Base de testeo
test_tweets1 = test_data['text'].tolist()
test_labels1 = test_data['name'].tolist()


cv = CountVectorizer()
cv.fit_transform(train_tweets1 + test_tweets1)


tfidf = TfidfTransformer()

train_features1 = tfidf.fit_transform(cv.transform(train_tweets1))
test_features1 = tfidf.transform(cv.transform(test_tweets1))

# Modelos

## Logit

In [ ]:
clf = LogisticRegression()
clf.fit(train_features, train_labels)

y_pred_logit = clf.predict(test_features)

In [ ]:
test['name'] = y_pred_logit

In [ ]:
test = test.drop(['text'], axis=1)

,id,name
0,6f4f09622255e9d9d76251b4,Lopez
1,aefd318f57227e4053f64191,Uribe
2,d1eb53be99fa263c10911af9,Petro
3,bb65250f1ff7582e4380a9c9,Uribe
4,0017570ee84b80e1c3e10218,Lopez
...,...,...
1495,4d1dc06d6ca8d0da93ceb42a,Uribe
1496,db282aa075d8ffb4dc9d72ae,Uribe
1497,afaac0a39ec1ea5f58308407,Uribe
1498,262b591834494af862ed6f09,Uribe


In [ ]:
test.to_csv('modelo_logit.csv', index=False)
test = test.drop(['name'], axis=1)

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(train_features, train_labels)

y_pred_RF = clf.predict(test_features)

test['name'] = y_pred_RF

test.to_csv('modelo_RF.csv', index=False)
test = test.drop(['name'], axis=1)

## LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

train_features1_dense = train_features1.toarray()
test_features1_dense = test_features1.toarray()

model = LinearDiscriminantAnalysis()

model.fit(train_features1_dense, train_labels1)

y_pred = model.predict(test_features1_dense)

from sklearn.metrics import accuracy_score
# Evaluar el modelo en el conjunto de prueba
accuracy = accuracy_score(test_labels1, y_pred)
print("Precisión del modelo:", accuracy)


Precisión del modelo: 0.7197860962566844


In [ ]:
train_features_dense = train_features.toarray()
test_features_dense = test_features.toarray()

# Entrenar modelo
model = LinearDiscriminantAnalysis()
model.fit(train_features_dense, train_labels)

# Predecir las categorías de los tweets en el conjunto de prueba
y_pred = model.predict(test_features_dense)

test['name'] = y_pred
test1 = test
test1 = test1.drop(['text'], axis=1)
test1.to_csv('modelo_LDA.csv', index=False)
test = test.drop(['name'], axis=1)

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Entrenar modelo de Naive Bayes
nb = MultinomialNB()
nb.fit(train_features, train_labels)

# Predecir las categorías de los tweets en el conjunto de prueba
y_pred = nb.predict(test_features)

test['name'] = y_pred
test.to_csv('modelo_NB.csv', index=False)
test = test.drop(['name'], axis=1)

## Neural Networks

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels1)

# Convertir las etiquetas en enteros utilizando el LabelEncoder
train_labels_encoded1 = le.transform(train_labels1)
train_features_dense1 = train_features1.toarray()
train_labels_encoded1 = np.array(train_labels_encoded1)

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras import regularizers

model = keras.Sequential([
    keras.layers.Dense(512, input_dim=train_features_dense1.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_features_dense1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(test_features_dense1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba:", accuracy)

Epoch 1/60
187/187 [==============================] - 38s 203ms/step - loss: 0.7033 - accuracy: 0.9291 - val_loss: 1.0323 - val_accuracy: 0.7928
Epoch 2/60
187/187 [==============================] - 45s 240ms/step - loss: 0.6863 - accuracy: 0.9382 - val_loss: 0.9895 - val_accuracy: 0.8035
Epoch 3/60
187/187 [==============================] - 45s 239ms/step - loss: 0.6920 - accuracy: 0.9326 - val_loss: 1.0047 - val_accuracy: 0.8182
Epoch 4/60
187/187 [==============================] - 44s 237ms/step - loss: 0.6832 - accuracy: 0.9365 - val_loss: 1.0169 - val_accuracy: 0.8082
Epoch 5/60
187/187 [==============================] - 40s 215ms/step - loss: 0.7002 - accuracy: 0.9325 - val_loss: 1.0388 - val_accuracy: 0.8061
Epoch 6/60
187/187 [==============================] - 40s 213ms/step - loss: 0.7116 - accuracy: 0.9325 - val_loss: 1.0097 - val_accuracy: 0.8102
Epoch 7/60
187/187 [==============================] - 41s 219ms/step - loss: 0.6754 - accuracy: 0.9375 - val_loss: 1.0178 - val_ac

In [ ]:
# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(test_features_dense1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba:", accuracy)

59/59 [==============================] - 2s 32ms/step - loss: 0.9819 - accuracy: 0.8209
Exactitud en el conjunto de prueba: 0.8208556175231934


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels1)

# Convertir las etiquetas en enteros utilizando el LabelEncoder
train_labels_encoded1 = le.transform(train_labels1)
train_features_dense1 = train_features1.toarray()
train_labels_encoded1 = np.array(train_labels_encoded1)

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

import tensorflow as tf
from tensorflow import keras

from keras import regularizers

model = keras.Sequential([
    keras.layers.Dense(512, input_dim=train_features_dense1.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.8),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.8),
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_features_dense1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(test_features_dense1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba:", accuracy)

Epoch 1/60
187/187 [==============================] - 39s 204ms/step - loss: 1.9324 - accuracy: 0.4148 - val_loss: 1.1767 - val_accuracy: 0.4813
Epoch 2/60
187/187 [==============================] - 36s 193ms/step - loss: 1.2483 - accuracy: 0.5875 - val_loss: 1.2863 - val_accuracy: 0.6070
Epoch 3/60
187/187 [==============================] - 37s 200ms/step - loss: 1.3137 - accuracy: 0.6121 - val_loss: 1.3292 - val_accuracy: 0.6136
Epoch 4/60
187/187 [==============================] - 36s 194ms/step - loss: 1.3254 - accuracy: 0.6201 - val_loss: 1.3454 - val_accuracy: 0.6156
Epoch 5/60
187/187 [==============================] - 38s 202ms/step - loss: 1.3324 - accuracy: 0.6286 - val_loss: 1.3772 - val_accuracy: 0.6003
Epoch 6/60
187/187 [==============================] - 38s 202ms/step - loss: 1.3398 - accuracy: 0.6400 - val_loss: 1.3548 - val_accuracy: 0.6096
Epoch 7/60
187/187 [==============================] - 37s 195ms/step - loss: 1.3300 - accuracy: 0.6420 - val_loss: 1.3419 - val_ac

In [ ]:
test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

import tensorflow as tf
from tensorflow import keras

from keras import regularizers

model = keras.Sequential([
    keras.layers.Dense(512, input_dim=train_features_dense1.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_features_dense1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(test_features_dense1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba:", accuracy)

Epoch 1/60
187/187 [==============================] - 50s 265ms/step - loss: 1.4298 - accuracy: 0.6196 - val_loss: 1.1380 - val_accuracy: 0.7353
Epoch 2/60
187/187 [==============================] - 36s 193ms/step - loss: 1.1414 - accuracy: 0.7510 - val_loss: 1.1923 - val_accuracy: 0.7400
Epoch 3/60
187/187 [==============================] - 40s 212ms/step - loss: 1.1627 - accuracy: 0.7602 - val_loss: 1.2001 - val_accuracy: 0.7634
Epoch 4/60
187/187 [==============================] - 36s 195ms/step - loss: 1.1735 - accuracy: 0.7690 - val_loss: 1.2478 - val_accuracy: 0.7567
Epoch 5/60
187/187 [==============================] - 40s 212ms/step - loss: 1.1723 - accuracy: 0.7780 - val_loss: 1.2104 - val_accuracy: 0.7594
Epoch 6/60
187/187 [==============================] - 36s 195ms/step - loss: 1.1747 - accuracy: 0.7814 - val_loss: 1.2655 - val_accuracy: 0.7433
Epoch 7/60
187/187 [==============================] - 38s 206ms/step - loss: 1.2049 - accuracy: 0.7747 - val_loss: 1.2131 - val_ac

In [ ]:
test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

import tensorflow as tf
from tensorflow import keras

from keras import regularizers

model = keras.Sequential([
    keras.layers.Dense(216, input_dim=train_features_dense1.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_features_dense1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(test_features_dense1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba:", accuracy)

Epoch 1/60
187/187 [==============================] - 14s 65ms/step - loss: 1.2267 - accuracy: 0.5842 - val_loss: 1.0893 - val_accuracy: 0.7326
Epoch 2/60
187/187 [==============================] - 11s 58ms/step - loss: 1.0583 - accuracy: 0.7630 - val_loss: 1.0592 - val_accuracy: 0.7547
Epoch 3/60
187/187 [==============================] - 11s 60ms/step - loss: 1.0327 - accuracy: 0.7799 - val_loss: 1.0498 - val_accuracy: 0.7587
Epoch 4/60
187/187 [==============================] - 11s 60ms/step - loss: 1.0157 - accuracy: 0.7882 - val_loss: 1.0354 - val_accuracy: 0.7988
Epoch 5/60
187/187 [==============================] - 10s 53ms/step - loss: 1.0065 - accuracy: 0.7964 - val_loss: 1.0516 - val_accuracy: 0.7787
Epoch 6/60
187/187 [==============================] - 11s 60ms/step - loss: 0.9937 - accuracy: 0.8008 - val_loss: 1.0345 - val_accuracy: 0.7828
Epoch 7/60
187/187 [==============================] - 11s 59ms/step - loss: 0.9848 - accuracy: 0.8133 - val_loss: 1.0462 - val_accuracy:

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras import regularizers


from sklearn.preprocessing import StandardScaler

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

scaler = StandardScaler()

train_features_scaled1 = scaler.fit_transform(train_features_dense1)
test_features_scaled1 = scaler.transform(test_features_dense1)

model = keras.Sequential([
    keras.layers.Dense(216, input_dim=train_features_dense1.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features_scaled1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba con datos normalizados
loss, accuracy = model.evaluate(test_features_scaled1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba con datos normalizados:", accuracy)

Epoch 1/60
187/187 [==============================] - 12s 63ms/step - loss: 6.5945 - accuracy: 0.5221 - val_loss: 3.3038 - val_accuracy: 0.6210
Epoch 2/60
187/187 [==============================] - 12s 63ms/step - loss: 1.9619 - accuracy: 0.8533 - val_loss: 1.6018 - val_accuracy: 0.8088
Epoch 3/60
187/187 [==============================] - 10s 52ms/step - loss: 0.9007 - accuracy: 0.9687 - val_loss: 1.0982 - val_accuracy: 0.8028
Epoch 4/60
187/187 [==============================] - 11s 59ms/step - loss: 0.5542 - accuracy: 0.9830 - val_loss: 1.0024 - val_accuracy: 0.7881
Epoch 5/60
187/187 [==============================] - 11s 61ms/step - loss: 0.5126 - accuracy: 0.9769 - val_loss: 1.1374 - val_accuracy: 0.7500
Epoch 6/60
187/187 [==============================] - 11s 61ms/step - loss: 0.5822 - accuracy: 0.9696 - val_loss: 1.3363 - val_accuracy: 0.7620
Epoch 7/60
187/187 [==============================] - 10s 52ms/step - loss: 0.7355 - accuracy: 0.9629 - val_loss: 1.4960 - val_accuracy:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout
from keras import regularizers


from sklearn.preprocessing import StandardScaler

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

scaler = StandardScaler()

train_features_scaled1 = scaler.fit_transform(train_features_dense1)
test_features_scaled1 = scaler.transform(test_features_dense1)

model = keras.Sequential([
    keras.layers.Dense(216, input_dim=train_features_dense1.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5), 
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features_scaled1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba con datos normalizados
loss, accuracy = model.evaluate(test_features_scaled1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba con datos normalizados:", accuracy)

Epoch 1/60
187/187 [==============================] - 20s 95ms/step - loss: 5.9796 - accuracy: 0.5604 - val_loss: 5.6251 - val_accuracy: 0.5943
Epoch 2/60
187/187 [==============================] - 12s 63ms/step - loss: 2.3833 - accuracy: 0.8188 - val_loss: 1.7605 - val_accuracy: 0.8088
Epoch 3/60
187/187 [==============================] - 12s 63ms/step - loss: 1.1502 - accuracy: 0.9622 - val_loss: 1.2800 - val_accuracy: 0.8148
Epoch 4/60
187/187 [==============================] - 11s 57ms/step - loss: 0.7848 - accuracy: 0.9794 - val_loss: 1.1071 - val_accuracy: 0.8115
Epoch 5/60
187/187 [==============================] - 11s 59ms/step - loss: 0.6651 - accuracy: 0.9788 - val_loss: 1.0920 - val_accuracy: 0.8001
Epoch 6/60
187/187 [==============================] - 12s 64ms/step - loss: 0.6753 - accuracy: 0.9736 - val_loss: 1.1705 - val_accuracy: 0.8015
Epoch 7/60
187/187 [==============================] - 12s 63ms/step - loss: 0.7530 - accuracy: 0.9666 - val_loss: 1.4585 - val_accuracy:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout
from keras import regularizers


from sklearn.preprocessing import StandardScaler

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

scaler = StandardScaler()

train_features_scaled1 = scaler.fit_transform(train_features_dense1)
test_features_scaled1 = scaler.transform(test_features_dense1)

model = keras.Sequential([
    keras.layers.Dense(216, input_dim=train_features_dense1.shape[1], activation='softmax', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5), 
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features_scaled1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba con datos normalizados
loss, accuracy = model.evaluate(test_features_scaled1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba con datos normalizados:", accuracy)

Epoch 1/60
187/187 [==============================] - 12s 61ms/step - loss: 1.2647 - accuracy: 0.3879 - val_loss: 1.1513 - val_accuracy: 0.3616
Epoch 2/60
187/187 [==============================] - 11s 61ms/step - loss: 1.1651 - accuracy: 0.3861 - val_loss: 1.1994 - val_accuracy: 0.3623
Epoch 3/60
187/187 [==============================] - 10s 52ms/step - loss: 1.1890 - accuracy: 0.4423 - val_loss: 1.1893 - val_accuracy: 0.6303
Epoch 4/60
187/187 [==============================] - 11s 59ms/step - loss: 1.1326 - accuracy: 0.7053 - val_loss: 1.1447 - val_accuracy: 0.7654
Epoch 5/60
187/187 [==============================] - 11s 61ms/step - loss: 1.0239 - accuracy: 0.7135 - val_loss: 1.0734 - val_accuracy: 0.7841
Epoch 6/60
187/187 [==============================] - 11s 59ms/step - loss: 0.9248 - accuracy: 0.7234 - val_loss: 1.0060 - val_accuracy: 0.8055
Epoch 7/60
187/187 [==============================] - 10s 51ms/step - loss: 0.8425 - accuracy: 0.7471 - val_loss: 0.9702 - val_accuracy:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout
from keras import regularizers


from sklearn.preprocessing import StandardScaler

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

scaler = StandardScaler()

train_features_scaled1 = scaler.fit_transform(train_features_dense1)
test_features_scaled1 = scaler.transform(test_features_dense1)

model = keras.Sequential([
    keras.layers.Dense(216, input_dim=train_features_dense1.shape[1], activation=tf.nn.elu, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5), 
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features_scaled1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba con datos normalizados
loss, accuracy = model.evaluate(test_features_scaled1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba con datos normalizados:", accuracy)

Epoch 1/60
187/187 [==============================] - 13s 65ms/step - loss: 5.0349 - accuracy: 0.5743 - val_loss: 5.3970 - val_accuracy: 0.6150
Epoch 2/60
187/187 [==============================] - 12s 63ms/step - loss: 1.7066 - accuracy: 0.9056 - val_loss: 1.5873 - val_accuracy: 0.7988
Epoch 3/60
187/187 [==============================] - 10s 54ms/step - loss: 0.8992 - accuracy: 0.9783 - val_loss: 1.1982 - val_accuracy: 0.7914
Epoch 4/60
187/187 [==============================] - 12s 62ms/step - loss: 0.6869 - accuracy: 0.9763 - val_loss: 1.2168 - val_accuracy: 0.7781
Epoch 5/60
187/187 [==============================] - 12s 62ms/step - loss: 0.7717 - accuracy: 0.9604 - val_loss: 1.6382 - val_accuracy: 0.7393
Epoch 6/60
187/187 [==============================] - 12s 62ms/step - loss: 0.9502 - accuracy: 0.9462 - val_loss: 1.7976 - val_accuracy: 0.7473
Epoch 7/60
187/187 [==============================] - 11s 58ms/step - loss: 1.0661 - accuracy: 0.9489 - val_loss: 1.7166 - val_accuracy:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout
from keras import regularizers


from sklearn.preprocessing import StandardScaler

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

scaler = StandardScaler()

train_features_scaled1 = scaler.fit_transform(train_features_dense1)
test_features_scaled1 = scaler.transform(test_features_dense1)

model = keras.Sequential([
    keras.layers.Dense(64, input_dim=train_features_dense1.shape[1], activation=tf.nn.elu, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5), 
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features_scaled1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba con datos normalizados
loss, accuracy = model.evaluate(test_features_scaled1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba con datos normalizados:", accuracy)

Epoch 1/60
187/187 [==============================] - 21s 27ms/step - loss: 3.6765 - accuracy: 0.5430 - val_loss: 2.3438 - val_accuracy: 0.6745
Epoch 2/60
187/187 [==============================] - 3s 18ms/step - loss: 1.4397 - accuracy: 0.8471 - val_loss: 1.2515 - val_accuracy: 0.8175
Epoch 3/60
187/187 [==============================] - 3s 18ms/step - loss: 0.7910 - accuracy: 0.9644 - val_loss: 1.0189 - val_accuracy: 0.8142
Epoch 4/60
187/187 [==============================] - 4s 23ms/step - loss: 0.5499 - accuracy: 0.9858 - val_loss: 0.8979 - val_accuracy: 0.8048
Epoch 5/60
187/187 [==============================] - 4s 22ms/step - loss: 0.4394 - accuracy: 0.9868 - val_loss: 0.8968 - val_accuracy: 0.7814
Epoch 6/60
187/187 [==============================] - 3s 18ms/step - loss: 0.4158 - accuracy: 0.9831 - val_loss: 1.0452 - val_accuracy: 0.7587
Epoch 7/60
187/187 [==============================] - 3s 17ms/step - loss: 0.4641 - accuracy: 0.9738 - val_loss: 1.1088 - val_accuracy: 0.770

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout
from keras import regularizers


from sklearn.preprocessing import StandardScaler

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

scaler = StandardScaler()

train_features_scaled1 = scaler.fit_transform(train_features_dense1)
test_features_scaled1 = scaler.transform(test_features_dense1)

model = keras.Sequential([
    keras.layers.Dense(152, input_dim=train_features_dense1.shape[1], activation=tf.nn.elu, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5), 
    keras.layers.Dense(64, input_dim=train_features_dense1.shape[1], activation=tf.nn.elu, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5), 
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features_scaled1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba con datos normalizados
loss, accuracy = model.evaluate(test_features_scaled1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba con datos normalizados:", accuracy)

Epoch 1/60
187/187 [==============================] - 8s 40ms/step - loss: 3.3305 - accuracy: 0.6156 - val_loss: 2.6974 - val_accuracy: 0.7774
Epoch 2/60
187/187 [==============================] - 8s 43ms/step - loss: 2.2616 - accuracy: 0.8700 - val_loss: 2.4218 - val_accuracy: 0.7774
Epoch 3/60
187/187 [==============================] - 7s 40ms/step - loss: 1.6806 - accuracy: 0.9489 - val_loss: 1.9423 - val_accuracy: 0.7854
Epoch 4/60
187/187 [==============================] - 8s 43ms/step - loss: 1.2525 - accuracy: 0.9701 - val_loss: 1.7232 - val_accuracy: 0.7761
Epoch 5/60
187/187 [==============================] - 8s 41ms/step - loss: 0.9991 - accuracy: 0.9751 - val_loss: 1.5020 - val_accuracy: 0.7848
Epoch 6/60
187/187 [==============================] - 8s 40ms/step - loss: 0.8667 - accuracy: 0.9738 - val_loss: 1.7272 - val_accuracy: 0.7239
Epoch 7/60
187/187 [==============================] - 8s 43ms/step - loss: 0.8949 - accuracy: 0.9627 - val_loss: 1.6865 - val_accuracy: 0.7513

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout
from keras import regularizers


from sklearn.preprocessing import StandardScaler

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

scaler = StandardScaler()

train_features_scaled1 = scaler.fit_transform(train_features_dense1)
test_features_scaled1 = scaler.transform(test_features_dense1)

model = keras.Sequential([
    keras.layers.Dense(152, input_dim=train_features_dense1.shape[1], activation=tf.nn.elu, kernel_regularizer=regularizers.l2(0.1)),
    Dropout(0.5), 
    keras.layers.Dense(64, input_dim=train_features_dense1.shape[1], activation=tf.nn.elu, kernel_regularizer=regularizers.l2(0.1)),
    Dropout(0.5), 
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features_scaled1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba con datos normalizados
loss, accuracy = model.evaluate(test_features_scaled1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba con datos normalizados:", accuracy)

Epoch 1/60
187/187 [==============================] - 10s 49ms/step - loss: 7.3077 - accuracy: 0.6895 - val_loss: 3.6353 - val_accuracy: 0.7774
Epoch 2/60
187/187 [==============================] - 7s 37ms/step - loss: 2.3924 - accuracy: 0.8947 - val_loss: 2.6178 - val_accuracy: 0.7513
Epoch 3/60
187/187 [==============================] - 9s 48ms/step - loss: 2.2187 - accuracy: 0.8731 - val_loss: 2.7452 - val_accuracy: 0.7553
Epoch 4/60
187/187 [==============================] - 7s 39ms/step - loss: 2.1472 - accuracy: 0.8822 - val_loss: 2.7872 - val_accuracy: 0.7320
Epoch 5/60
187/187 [==============================] - 8s 44ms/step - loss: 2.1963 - accuracy: 0.8848 - val_loss: 2.7750 - val_accuracy: 0.7754
Epoch 6/60
187/187 [==============================] - 8s 41ms/step - loss: 2.2769 - accuracy: 0.8873 - val_loss: 3.0223 - val_accuracy: 0.7507
Epoch 7/60
187/187 [==============================] - 8s 42ms/step - loss: 2.5823 - accuracy: 0.8904 - val_loss: 3.5596 - val_accuracy: 0.738

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels1)

# Convertir las etiquetas en enteros utilizando el LabelEncoder
train_labels_encoded1 = le.transform(train_labels1)
train_features_dense1 = train_features1.toarray()
train_labels_encoded1 = np.array(train_labels_encoded1)

test_labels_encoded1 = le.transform(test_labels1)
test_features_dense1 = test_features1.toarray()
test_labels_encoded1 = np.array(test_labels_encoded1)

import tensorflow as tf
from tensorflow import keras

from keras import regularizers

model = keras.Sequential([
    keras.layers.Dense(120, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.8),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.8),
    keras.layers.Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_features_dense1, tf.keras.utils.to_categorical(train_labels_encoded1), 
          epochs = 60, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(test_features_dense1, tf.keras.utils.to_categorical(test_labels_encoded1))

print("Exactitud en el conjunto de prueba:", accuracy)

Epoch 1/60
187/187 [==============================] - 8s 40ms/step - loss: 1.4309 - accuracy: 0.3769 - val_loss: 1.1392 - val_accuracy: 0.3509
Epoch 2/60
187/187 [==============================] - 6s 30ms/step - loss: 1.1236 - accuracy: 0.4250 - val_loss: 1.1244 - val_accuracy: 0.5635
Epoch 3/60
187/187 [==============================] - 7s 39ms/step - loss: 1.1408 - accuracy: 0.5268 - val_loss: 1.1285 - val_accuracy: 0.5862
Epoch 4/60
187/187 [==============================] - 6s 30ms/step - loss: 1.1586 - accuracy: 0.5768 - val_loss: 1.1396 - val_accuracy: 0.6096
Epoch 5/60
187/187 [==============================] - 6s 35ms/step - loss: 1.1574 - accuracy: 0.5949 - val_loss: 1.1322 - val_accuracy: 0.5989
Epoch 6/60
187/187 [==============================] - 6s 33ms/step - loss: 1.1532 - accuracy: 0.5969 - val_loss: 1.1375 - val_accuracy: 0.6150
Epoch 7/60
187/187 [==============================] - 6s 30ms/step - loss: 1.1498 - accuracy: 0.6020 - val_loss: 1.1286 - val_accuracy: 0.6110

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels)

# Convertir las etiquetas en enteros utilizando el LabelEncoder
train_labels_encoded = le.transform(train_labels)

train_features_dense = train_features.toarray()
train_labels_encoded = np.array(train_labels_encoded)

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras import regularizers

model = keras.Sequential([
    keras.layers.Dense(512, input_dim=train_features_dense1.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation='softmax')
])

In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
model.fit(train_features_dense, tf.keras.utils.to_categorical(train_labels_encoded), 
          epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
234/234 [==============================] - 52s 216ms/step - loss: 1.7731 - accuracy: 0.5097 - val_loss: 1.2270 - val_accuracy: 0.5957
Epoch 2/50
234/234 [==============================] - 51s 217ms/step - loss: 1.2003 - accuracy: 0.6181 - val_loss: 1.2093 - val_accuracy: 0.5979
Epoch 3/50
234/234 [==============================] - 49s 210ms/step - loss: 1.1650 - accuracy: 0.6247 - val_loss: 1.1907 - val_accuracy: 0.6198
Epoch 4/50
234/234 [==============================] - 51s 217ms/step - loss: 1.1366 - accuracy: 0.6390 - val_loss: 1.1733 - val_accuracy: 0.6241
Epoch 5/50
234/234 [==============================] - 49s 210ms/step - loss: 1.1133 - accuracy: 0.6462 - val_loss: 1.1197 - val_accuracy: 0.6465
Epoch 6/50
234/234 [==============================] - 52s 220ms/step - loss: 1.0690 - accuracy: 0.6565 - val_loss: 1.1037 - val_accuracy: 0.6396
Epoch 7/50
234/234 [==============================] - 49s 210ms/step - loss: 1.0478 - accuracy: 0.6683 - val_loss: 1.0847 - val_ac

In [ ]:
test

,id,text
0,6f4f09622255e9d9d76251b4,"Le ruego a la Procuradora General @PGN_COL, qu..."
1,aefd318f57227e4053f64191,Más de $51.000 millones de pesos serán inverti...
2,d1eb53be99fa263c10911af9,Luchar contra la crisis climática implica camb...
3,bb65250f1ff7582e4380a9c9,1. Hoy en #OpinionesCruzadas de @rcnradio reté...
4,0017570ee84b80e1c3e10218,Ante amenazas del ELN y sistemática reducción ...
...,...,...
1495,4d1dc06d6ca8d0da93ceb42a,Pido perdón - Periódico Debate Senadora Paola ...
1496,db282aa075d8ffb4dc9d72ae,"Dos estilos políticos, dos visiones de país:\n..."
1497,afaac0a39ec1ea5f58308407,"#Urgente, nos informan que una compañera del p..."
1498,262b591834494af862ed6f09,Empresas avícolas del Cauca no pueden sacar lo...


In [ ]:
# Predecir las categorías de los tweets en el conjunto de prueba
test_features_dense = test_features.toarray()

Autor = {1: 'Petro', 2: 'Uribe', 0: 'Lopez'}

y_pred = model.predict(test_features_dense)
y_pred = np.argmax(y_pred, axis=1)

y_pred = [Autor[i] for i in y_pred]

test['name'] = y_pred
test_envio = test.drop(['text'], axis=1)
test = test.drop(['name'], axis=1)
test_envio.to_csv('modelo_Redes.csv', index=False)

47/47 [==============================] - 1s 20ms/step


In [ ]:
test

,id,text
0,6f4f09622255e9d9d76251b4,"Le ruego a la Procuradora General @PGN_COL, qu..."
1,aefd318f57227e4053f64191,Más de $51.000 millones de pesos serán inverti...
2,d1eb53be99fa263c10911af9,Luchar contra la crisis climática implica camb...
3,bb65250f1ff7582e4380a9c9,1. Hoy en #OpinionesCruzadas de @rcnradio reté...
4,0017570ee84b80e1c3e10218,Ante amenazas del ELN y sistemática reducción ...
...,...,...
1495,4d1dc06d6ca8d0da93ceb42a,Pido perdón - Periódico Debate Senadora Paola ...
1496,db282aa075d8ffb4dc9d72ae,"Dos estilos políticos, dos visiones de país:\n..."
1497,afaac0a39ec1ea5f58308407,"#Urgente, nos informan que una compañera del p..."
1498,262b591834494af862ed6f09,Empresas avícolas del Cauca no pueden sacar lo...


In [ ]:
test_envio

,id,name
0,6f4f09622255e9d9d76251b4,Petro
1,aefd318f57227e4053f64191,Uribe
2,d1eb53be99fa263c10911af9,Petro
3,bb65250f1ff7582e4380a9c9,Uribe
4,0017570ee84b80e1c3e10218,Lopez
...,...,...
1495,4d1dc06d6ca8d0da93ceb42a,Uribe
1496,db282aa075d8ffb4dc9d72ae,Uribe
1497,afaac0a39ec1ea5f58308407,Uribe
1498,262b591834494af862ed6f09,Uribe


In [ ]:
test.to_csv('modelo_Redes.csv', index=False)

## Xgboost

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!pip install --upgrade xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(train_tweets1)
train_features1 = vectorizer.transform(train_tweets1)

vectorizer = TfidfVectorizer()
vectorizer.fit(test_tweets1)
test_features1 = vectorizer.transform(test_tweets1)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels1_encoder = label_encoder.fit_transform(train_labels1)
test_labels1_encoder = label_encoder.fit_transform(test_labels1)

dtrain = xgb.DMatrix(train_features1, label=train_labels1_encoder)
dtest = xgb.DMatrix(test_features1, label=test_labels1_encoder)

params = {
    'max_depth': 1500,
    'objective': 'multi:softmax',
    'num_class': len(train_data['name'].unique()) 
}

model = xgb.train(params, dtrain, num_boost_round=10)

from sklearn.metrics import accuracy_score

y_pred = model.predict(dtest)

from sklearn.metrics import accuracy_score
# Evaluar el modelo en el conjunto de prueba
accuracy = accuracy_score(test_labels1_encoder, y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.32192513368983955


In [ ]:
y_pred

array([2., 2., 2., ..., 2., 2., 2.], dtype=float32)

In [ ]:
test_labels1

['Petro',
 'Uribe',
 'Petro',
 'Uribe',
 'Lopez',
 'Petro',
 'Uribe',
 'Lopez',
 'Lopez',
 'Uribe',
 'Uribe',
 'Lopez',
 'Lopez',
 'Petro',
 'Uribe',
 'Lopez',
 'Lopez',
 'Lopez',
 'Lopez',
 'Uribe',
 'Petro',
 'Uribe',
 'Petro',
 'Petro',
 'Petro',
 'Lopez',
 'Lopez',
 'Lopez',
 'Lopez',
 'Uribe',
 'Lopez',
 'Uribe',
 'Lopez',
 'Lopez',
 'Uribe',
 'Lopez',
 'Lopez',
 'Petro',
 'Uribe',
 'Uribe',
 'Uribe',
 'Uribe',
 'Uribe',
 'Lopez',
 'Uribe',
 'Petro',
 'Lopez',
 'Uribe',
 'Uribe',
 'Petro',
 'Uribe',
 'Uribe',
 'Uribe',
 'Petro',
 'Lopez',
 'Uribe',
 'Petro',
 'Lopez',
 'Lopez',
 'Uribe',
 'Lopez',
 'Petro',
 'Lopez',
 'Lopez',
 'Lopez',
 'Lopez',
 'Petro',
 'Lopez',
 'Petro',
 'Lopez',
 'Petro',
 'Petro',
 'Lopez',
 'Lopez',
 'Lopez',
 'Lopez',
 'Uribe',
 'Uribe',
 'Lopez',
 'Lopez',
 'Petro',
 'Lopez',
 'Lopez',
 'Lopez',
 'Lopez',
 'Petro',
 'Petro',
 'Uribe',
 'Petro',
 'Uribe',
 'Uribe',
 'Uribe',
 'Petro',
 'Lopez',
 'Uribe',
 'Petro',
 'Lopez',
 'Petro',
 'Uribe',
 'Petro',


In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(train_tweets)

train_features = vectorizer.transform(train_tweets)
test_features = vectorizer.transform(test_tweets)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_data['name']= label_encoder.fit_transform(train_data['name'])
test_data['name']= label_encoder.fit_transform(test_data['name'])

In [ ]:
dtrain = xgb.DMatrix(train_features, label=train_data['name'])
dtest = xgb.DMatrix(test_features, label=test_data['name'])

In [ ]:
params = {
    'max_depth': 70,
    'objective': 'multi:softmax',
    'num_class': len(train_data['name'].unique()) 
}

model = xgb.train(params, dtrain, num_boost_round=10)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(dtest)

accuracy = accuracy_score(test_data['name'], y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.7374331550802139


# Hyperparameter tunning 

In [ ]:
from sklearn.model_selection import GridSearchCV

## Logit

In [ ]:
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': np.logspace(-4, 4, 20),
    'solver': ['saga', 'liblinear']
}

model = LogisticRegression(random_state=42)

grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(train_features, train_labels)

print("Mejores hiperparámetros: ", grid_search.best_params_)
print("Score: ", grid_search.best_score_)

Mejores hiperparámetros:  {'C': 11.288378916846883, 'penalty': 'l2', 'solver': 'liblinear'}
Score:  0.8423354305971623


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
200 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py", line 1291, in fit
    fold_coefs_ = Parallel(n_jobs=self.n_jobs, verbose=self.verbose, prefer=prefer)(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/utils/parallel.py", line 63, in __call__
    return super().__call__(itera

In [ ]:
clf = LogisticRegression(C = 11.288378916846883, 
                         penalty = 'l2', 
                         solver = 'liblinear')

clf.fit(train_features, train_labels)

y_pred_logit = clf.predict(test_features)

test['name'] = y_pred_logit

test.to_csv('modelo_logit_hyp.csv', index=False)
test = test.drop(['name'], axis=1)